# 제 2회 기출문제

## 문제 1. 
주어진 데이터에서 'views' 컬럼을 기준으로 내림차순 정렬하고,<br>
'views' 컬럼 상위 10개 데이터를 'views' 컬럼 상위 10번째 값으로 대체한 후 <br>
'age' 컬럼이 80 이상인 데이터의 'views' 컬럼 평균값을 구하시오.(반올림하여 소수 둘째 자리까지 계산)



In [9]:
import pandas as pd

df = pd.read_csv('./ch2_data/members.csv')
df

,id,age,city,f1,f2,f3,f4,f5,subscribed,views
0,id01,2.0,서울,NaN,0,gold,ENFJ,91.297791,2021-07-16,6820.0
1,id02,9.0,서울,70.0,1,NaN,ENFJ,60.339826,2021-05-12,2534.0
2,id03,27.0,서울,61.0,1,gold,ISTJ,17.252986,2021-03-16,7312.0
3,id04,75.0,서울,NaN,2,NaN,INFP,52.667078,2021-07-21,493.0
4,id05,24.0,서울,85.0,2,NaN,ISFJ,29.269869,2021-03-07,1338.0
...,...,...,...,...,...,...,...,...,...,...
95,id95,77.0,경기,43.0,1,gold,INTJ,91.297791,2021-05-21,8697.0
96,id96,92.0,경기,53.0,1,silver,ENTJ,52.667078,2021-05-06,4336.0
97,id97,100.0,경기,NaN,0,gold,INFP,67.886373,2021-03-18,6687.0
98,id98,39.0,경기,58.0,2,NaN,INFP,98.429899,2021-10-02,865.0


In [12]:
# 1. views 컬럼 내림차순 정렬

df.sort_values('views', ascending=False, inplace=True)

# 2. 상위 10개 중에서 10번째 값 최솟값 구하기
min_value = df['views'][:10].min()

# 3. 상위 10개 중에서 10번째 값(최솟값) 대체
df.iloc[:10,-1] = min_value

# 4. age가 80이상의 views 컬럼 평균
conf = df['age']>=80
result = df[conf]['views'].mean()
print(round(result, 2))

5674.04


## 문제 2. 
주어진 데이터셋에서 앞에서부터 순서대로 80% 데이터만 활용해 <br>
'f1' 컬럼 결측치를 중앙 값으로 채우기 전후의 표준편차를 구하고 두 표본 표준편차의 차이를 계산하시오<br>
(두 표본 표준편차는 절대값으로 계산하고, 반올림하여 소수 둘째 자리까지 계산)


In [13]:
import pandas as pd
df = pd.read_csv('./ch2_data/members.csv')

In [16]:
line = int(len(df)*0.8)

df = df.iloc[:line]

In [18]:
std1 = df['f1'].std()
med = df['f1'].median()
df['f1'] = df['f1'].fillna(med)
std2 = df['f1'].std()

result = abs(std1-std2)
print(round(result,2))

3.56


## 문제 3.
주어진 데이터셋 'age' 컬럼의 이상치를 모두 더하시오.<br>
이상치는 평균으로부터 '표준편차*1.5'를 벗어나는 영역을 이상치라고 판단


In [19]:
import pandas as pd
df = pd.read_csv('./ch2_data/members.csv')

In [24]:
std1 = df['age'].std()
mean = df['age'].mean()

lower = mean - (std1 * 1.5)
upper = mean + (std1 * 1.5)
cond1 = df['age']<lower
cond2 = df['age']>upper

print(df[cond1|cond2]['age'].sum())

473.5


## 문제 4.
제품 배송 시간에 맞춰 배송되지 않을 확률값을 예측하시오.
- 예측할 컬럼 : Reached.on.Time_Y.N(0: 정시 도착, 1: 정시 도착 x)
- 평가 기준 : ROC_AUC로 평가


In [26]:
import pandas as pd

X_test = pd.read_csv('./ch2_data/X_test.csv')
X_train = pd.read_csv('./ch2_data/X_train.csv')
y_train = pd.read_csv('./ch2_data/y_train.csv')


In [29]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(8799, 11)
(2200, 11)
(8799, 2)


In [32]:
X_train.isnull().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64

In [33]:
X_test.isnull().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64

In [34]:
y_train['Reached.on.Time_Y.N'].value_counts()

Reached.on.Time_Y.N
1    5236
0    3563
Name: count, dtype: int64

In [38]:
from sklearn.preprocessing import LabelEncoder

cols = X_train.select_dtypes(include='object').columns
 

for col in cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])


In [39]:
X_train = X_train.drop('ID',axis=1)

In [40]:
X_test_id = X_test.pop('ID')

In [47]:
from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    X_train,
    y_train['Reached.on.Time_Y.N'],
    test_size=0.2,
    random_state=0
)


X_tr.shape,X_val.shape,y_tr.shape,y_val.shape

((7039, 10), (1760, 10), (7039,), (1760,))

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [49]:
# 로지스틱회귀
lr = LogisticRegression(random_state=0)
lr.fit(X_tr,y_tr)
pred = lr.predict_proba(X_val)
print(roc_auc_score(y_val, pred[:,1]))

0.7294862508383635


/Users/gihyun/Desktop/dev/GitHub/-BigDataAnalyst/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [50]:
dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_tr,y_tr)
pred = dt.predict_proba(X_val)
print(roc_auc_score(y_val,pred[:,1]))

0.6540107310529846


In [51]:
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr,y_tr)
pred = rf.predict_proba(X_val)
print(roc_auc_score(y_val,pred[:,1]))

0.7657780013413816


In [53]:
pred = rf.predict_proba(X_test)
pred

array([[0.61, 0.39],
       [0.61, 0.39],
       [0.67, 0.33],
       ...,
       [0.58, 0.42],
       [0.66, 0.34],
       [0.4 , 0.6 ]], shape=(2200, 2))

In [54]:
submit = pd.DataFrame({
    'ID':X_test_id,
    'Reached.on.Time_Y.N':pred[:,1]
})

In [55]:
submit.head(5)

,ID,Reached.on.Time_Y.N
0,8285,0.39
1,10192,0.39
2,8675,0.33
3,5753,0.42
4,448,0.97


### 심화학습


In [70]:
import pandas as pd

X_test = pd.read_csv('./ch2_data/X_test.csv')
X_train = pd.read_csv('./ch2_data/X_train.csv')
y_train = pd.read_csv('./ch2_data/y_train.csv')


In [71]:
from sklearn.preprocessing import LabelEncoder
cols = X_train.select_dtypes(include='object').columns

for col in cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])


In [72]:
X_train = X_train.drop('ID',axis=1)
X_test_id = X_test.pop('ID')

In [76]:
from sklearn.preprocessing import MinMaxScaler

cols=['Customer_care_calls','Customer_rating','Cost_of_the_Product',
      'Prior_purchases','Discount_offered','Weight_in_gms'
      ]

scaler = MinMaxScaler()
X_train[cols] = scaler.fit_transform(X_train[cols])
X_test[cols] = scaler.fit_transform(X_test[cols])


In [77]:
from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    X_train,
    y_train['Reached.on.Time_Y.N'],
    test_size=0.2,
    random_state=0
)


X_tr.shape,X_val.shape,y_tr.shape,y_val.shape

((7039, 10), (1760, 10), (7039,), (1760,))

In [78]:
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr,y_tr)
pred = rf.predict_proba(X_val)
print(roc_auc_score(y_val,pred[:,1]))

0.7657256874580818


# 제 3회 기출문제


## 1. 유형

### 문제1.
결측치가 있는 데이터의 행을 제거하고 앞에서부터 '70%' 데이터만 활용해<br>
'f1' 컬럼 1사분위 값을 구하시오.(소수점 제거 후 정수 출력)

In [90]:
import pandas as pd
df = pd.read_csv('./ch3_data/members.csv')

df = df.dropna()
lines = int(len(df)*0.7)
df = df.iloc[:lines]

print(int(df['f1'].quantile(.25)))

57


### 문제2.
2000년 데이터 중 2000년 평균보다 큰 값의 데이터 수를 구하시오
(index 연도)


In [91]:
import pandas as pd
df = pd.read_csv('./ch3_data/year.csv',index_col='Unnamed: 0')

In [ ]:
# 풀이 1. 행을 기준으로 평균 계산

m = df.loc[2000].mean()

print(sum(df.loc[2000,:]>m))

In [105]:
# 풀이 2. 행과 열을 변경해 평균 계산

df = df.T

m = df[2000].mean()
print(len(df[df[2000]>m]))

100


### 문제 3. 
결측치가 가장 많은 컬럼명을 출력하시오.

In [106]:
import pandas as pd
df = pd.read_csv("./ch3_data/members.csv")

In [109]:
df_cntNull = df.isnull().sum()

df_cntNull = df_cntNull.sort_values(ascending=False)
print(df_cntNull.index[0])

f1


### 문제 4. 
여행 보험 패키지 상품을 구매할 확률값을 예측하시오.<br>
예측할 컬럼 : Travellnsurance(패키지 구매 여부 0:구매 x, 1: 구매함) <br>
모델 성능은 roc-auc 평가지표

In [110]:
import pandas as pd
train = pd.read_csv('./ch3_data/train.csv')
test = pd.read_csv('./ch3_data/test.csv')

In [111]:
train.shape,test.shape

((1490, 10), (497, 9))

In [112]:
# train 결측치 x
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           1490 non-null   int64 
 1   Age                  1490 non-null   int64 
 2   Employment Type      1490 non-null   object
 3   GraduateOrNot        1490 non-null   object
 4   AnnualIncome         1490 non-null   int64 
 5   FamilyMembers        1490 non-null   int64 
 6   ChronicDiseases      1490 non-null   int64 
 7   FrequentFlyer        1490 non-null   object
 8   EverTravelledAbroad  1490 non-null   object
 9   TravelInsurance      1490 non-null   int64 
dtypes: int64(6), object(4)
memory usage: 116.5+ KB


In [114]:
# test 결측치 x
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           497 non-null    int64 
 1   Age                  497 non-null    int64 
 2   Employment Type      497 non-null    object
 3   GraduateOrNot        497 non-null    object
 4   AnnualIncome         497 non-null    int64 
 5   FamilyMembers        497 non-null    int64 
 6   ChronicDiseases      497 non-null    int64 
 7   FrequentFlyer        497 non-null    object
 8   EverTravelledAbroad  497 non-null    object
dtypes: int64(5), object(4)
memory usage: 35.1+ KB


In [115]:
train['TravelInsurance'].value_counts()

TravelInsurance
0    965
1    525
Name: count, dtype: int64

In [116]:
train.describe(include='object')

,Employment Type,GraduateOrNot,FrequentFlyer,EverTravelledAbroad
count,1490,1490,1490,1490
unique,2,2,2,2
top,Private Sector/Self Employed,Yes,No,No
freq,1056,1270,1175,1209


In [118]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           1490 non-null   int64 
 1   Age                  1490 non-null   int64 
 2   Employment Type      1490 non-null   object
 3   GraduateOrNot        1490 non-null   object
 4   AnnualIncome         1490 non-null   int64 
 5   FamilyMembers        1490 non-null   int64 
 6   ChronicDiseases      1490 non-null   int64 
 7   FrequentFlyer        1490 non-null   object
 8   EverTravelledAbroad  1490 non-null   object
 9   TravelInsurance      1490 non-null   int64 
dtypes: int64(6), object(4)
memory usage: 116.5+ KB


In [119]:
target = train.pop('TravelInsurance')

In [120]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [121]:
from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    train,
    target,
    test_size=0.2,
    random_state=0
)
X_tr.shape,X_val.shape,y_tr.shape,y_val.shape

((1192, 13), (298, 13), (1192,), (298,))

In [123]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [124]:
lr = LogisticRegression(random_state=0)

lr.fit(X_tr,y_tr)
pred = lr.predict_proba(X_val)
print(roc_auc_score(y_val,pred[:,1]))

0.7745126705653022


/Users/gihyun/Desktop/dev/GitHub/-BigDataAnalyst/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [126]:
dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_tr,y_tr)
pred = dt.predict_proba(X_val)
print(roc_auc_score(y_val, pred[:,1]))

0.7090155945419103


In [127]:
rf = RandomForestClassifier(random_state=0)

rf.fit(X_tr,y_tr)
pred = rf.predict_proba(X_val)
print(roc_auc_score(y_val,pred[:,1]))

0.850682261208577


In [128]:
pred = rf.predict_proba(test)
pred

array([[0.76, 0.24],
       [0.93, 0.07],
       [0.89, 0.11],
       [0.02, 0.98],
       [0.72, 0.28],
       [0.  , 1.  ],
       [0.53, 0.47],
       [0.93, 0.07],
       [0.22, 0.78],
       [0.95, 0.05],
       [0.93, 0.07],
       [0.92, 0.08],
       [0.7 , 0.3 ],
       [0.93, 0.07],
       [0.09, 0.91],
       [0.86, 0.14],
       [0.58, 0.42],
       [0.85, 0.15],
       [0.6 , 0.4 ],
       [0.38, 0.62],
       [0.88, 0.12],
       [0.88, 0.12],
       [0.  , 1.  ],
       [0.04, 0.96],
       [1.  , 0.  ],
       [0.01, 0.99],
       [0.01, 0.99],
       [0.86, 0.14],
       [0.02, 0.98],
       [0.01, 0.99],
       [0.91, 0.09],
       [0.73, 0.27],
       [0.9 , 0.1 ],
       [0.68, 0.32],
       [0.88, 0.12],
       [0.13, 0.87],
       [0.87, 0.13],
       [0.82, 0.18],
       [0.04, 0.96],
       [0.79, 0.21],
       [0.91, 0.09],
       [1.  , 0.  ],
       [0.69, 0.31],
       [0.74, 0.26],
       [0.08, 0.92],
       [0.86, 0.14],
       [0.02, 0.98],
       [0.74,

In [131]:
submit = pd.DataFrame({
    'index':test.index,
    'y_pred':pred[:,1]})

In [133]:
# submit.to_csv('./ch3_data/result.csv',index=False)

#### 심화 
기존 점수 : 0.85
RandomForestClassifier

In [ ]:
import pandas as pd

train = pd.read_csv('./ch3_data/train.csv')
test = pd.read_csv('./ch3_data/test.csv')

In [135]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           1490 non-null   int64 
 1   Age                  1490 non-null   int64 
 2   Employment Type      1490 non-null   object
 3   GraduateOrNot        1490 non-null   object
 4   AnnualIncome         1490 non-null   int64 
 5   FamilyMembers        1490 non-null   int64 
 6   ChronicDiseases      1490 non-null   int64 
 7   FrequentFlyer        1490 non-null   object
 8   EverTravelledAbroad  1490 non-null   object
 9   TravelInsurance      1490 non-null   int64 
dtypes: int64(6), object(4)
memory usage: 116.5+ KB


In [136]:
target = train.pop('TravelInsurance')

In [138]:
from sklearn.preprocessing import LabelEncoder

cols = train.select_dtypes(include='O').columns
for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])


In [140]:
from sklearn.model_selection import train_test_split

X_tr,X_val,y_tr,y_val = train_test_split(
    train,target,test_size=0.2, random_state=0
)
X_tr.shape,X_val.shape,y_tr.shape,y_val.shape

((1192, 9), (298, 9), (1192,), (298,))

In [142]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

rf = RandomForestClassifier(random_state=0)

rf.fit(X_tr,y_tr)
pred = rf.predict_proba(X_val)
print(roc_auc_score(y_val,pred[:,1]))

0.8630847953216375


#### 심화 
기존 점수 : 0.85 RandomForestClassifier<br>
LabelEncoder -> 0.8630847953216375<br>
스케일링(로버스트) : 0.8631091617933725<br>


In [145]:
import pandas as pd

train = pd.read_csv('./ch3_data/train.csv')
test = pd.read_csv('./ch3_data/test.csv')
target = train.pop('TravelInsurance')


from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
cols =['Age','AnnualIncome','FamilyMembers','ChronicDiseases']

train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])


from sklearn.preprocessing import LabelEncoder

cols = train.select_dtypes(include='O').columns
for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])


from sklearn.model_selection import train_test_split

X_tr,X_val,y_tr,y_val = train_test_split(
    train,target,test_size=0.2, random_state=0
)
X_tr.shape,X_val.shape,y_tr.shape,y_val.shape


from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

rf = RandomForestClassifier(random_state=0)

rf.fit(X_tr,y_tr)
pred = rf.predict_proba(X_val)
print(roc_auc_score(y_val,pred[:,1]))

0.8631091617933725


In [148]:
pred = rf.predict_proba(test)
submit = pd.DataFrame({
    'index':test.index,
    'pred':pred[:,1]})

# 제 4회 기출문제


## 1유형


### 문제
'age' 컬럼의 3사분위수와 1분위수의 차이를 절대값으로 구하고, 정수로 출력


In [150]:
import pandas as pd
df = pd.read_csv('./ch4_data/data4-1.csv')

In [152]:
q1 = df['age'].quantile(0.25)
q3 = df['age'].quantile(0.75)


In [154]:
print(int(abs(q1-q3)))

50


### 문제

모든 reactions 중 loves 와 wows를 합한 반응 비율이 40%보다 크고 50%보다 작은 데이터를 찾고<br>
이 중 type컬럼이 'video'인 데이터의 개수를 구하시오.

In [155]:
import pandas as pd
df = pd.read_csv('./ch4_data/data4-2.csv')
df

,id,type,reactions,comments,shares,likes,loves,wows,hahas,sads,angrys
0,1,video,529,512,262,432,92,3,1,1,0
1,2,photo,150,0,0,150,0,0,0,0,0
2,3,video,227,236,57,204,21,1,1,0,0
3,4,photo,111,0,0,111,0,0,0,0,0
4,5,photo,213,0,0,204,9,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
7045,7046,photo,89,0,0,89,0,0,0,0,0
7046,7047,photo,16,0,0,14,1,0,1,0,0
7047,7048,photo,2,0,0,1,1,0,0,0,0
7048,7049,photo,351,12,22,349,2,0,0,0,0


In [159]:
cond1 = (df['loves']+df['wows'])/df['reactions']>0.4
cond2 = (df['loves']+df['wows'])/df['reactions']<0.5
cond3 = df['type']=='video'

In [160]:
print(len(df[cond1&cond2&cond3]))

90


### 문제
'data_added' 컬럼의 날짜가 2018년 1월이면서 'country'에서 United Kingdom 단독 제작인 데이터의 개수

In [177]:
import pandas as pd
df = pd.read_csv("./ch4_data/data4-3.csv")
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries"
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act..."
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV"
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ..."
...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers"
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies"
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies"
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies"


In [188]:
df['date_added'] = pd.to_datetime(df['date_added'])

In [191]:
df['year'] = df['date_added'].dt.year
df['month'] = df['date_added'].dt.month

In [194]:
cond1 = df['country'] == 'United Kingdom'
cond2 = df['date_added'] >='2018-1-1'
cond3 = df['date_added'] <='2018-1-31'

print(len(df[cond1&cond2&cond3]))

6


## 2유형

### 문제
고객의 개인 정보와 행동 패턴을 기반으로 고객의 세분화 결과를 예측하시오.
- 예측할 컬럼 : Segmentation(1,2,3,4)
- 제출파일
    - ID : 고객 식별자
    - Segmentation(1,2,3,4)
- 모델의 성능 macro F1 score

In [195]:
import pandas as pd
train = pd.read_csv('./ch4_data/train.csv')
test = pd.read_csv('./ch4_data/test.csv')

In [ ]:
train.info()
# 결측치 x
# 범주형 6개
# 수치형 5개

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               6665 non-null   int64  
 1   Gender           6665 non-null   object 
 2   Ever_Married     6665 non-null   object 
 3   Age              6665 non-null   int64  
 4   Graduated        6665 non-null   object 
 5   Profession       6665 non-null   object 
 6   Work_Experience  6665 non-null   float64
 7   Spending_Score   6665 non-null   object 
 8   Family_Size      6665 non-null   float64
 9   Var_1            6665 non-null   object 
 10  Segmentation     6665 non-null   int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 572.9+ KB


In [ ]:
test.info()

# 결측치 x
# 범주형 ; 6
# 수치형 : 4

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2154 entries, 0 to 2153
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               2154 non-null   int64  
 1   Gender           2154 non-null   object 
 2   Ever_Married     2154 non-null   object 
 3   Age              2154 non-null   int64  
 4   Graduated        2154 non-null   object 
 5   Profession       2154 non-null   object 
 6   Work_Experience  2154 non-null   float64
 7   Spending_Score   2154 non-null   object 
 8   Family_Size      2154 non-null   float64
 9   Var_1            2154 non-null   object 
dtypes: float64(2), int64(2), object(6)
memory usage: 168.4+ KB


In [198]:
train.describe()

,ID,Age,Work_Experience,Family_Size,Segmentation
count,6665.00000,6665.000000,6665.000000,6665.000000,6665.000000
mean,463519.84096,43.536084,2.629107,2.841110,2.542836
std,2566.43174,16.524054,3.405365,1.524743,1.122723
min,458982.00000,18.000000,0.000000,1.000000,1.000000
25%,461349.00000,31.000000,0.000000,2.000000,2.000000
50%,463575.00000,41.000000,1.000000,2.000000,3.000000
75%,465741.00000,53.000000,4.000000,4.000000,4.000000
max,467974.00000,89.000000,14.000000,9.000000,4.000000


In [ ]:
train.describe(include='O')

,Gender,Ever_Married,Graduated,Profession,Spending_Score,Var_1
count,6665,6665,6665,6665,6665,6665
unique,2,2,2,9,3,7
top,Male,Yes,Yes,Artist,Low,Cat_6
freq,3677,3944,4249,2192,3999,4476


In [200]:
target = train.pop("Segmentation")

In [201]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    train,target, test_size=0.2, random_state=0
)

X_tr.shape, X_val.shape,y_tr.shape,y_val.shape

((5332, 29), (1333, 29), (5332,), (1333,))

In [202]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [ ]:
# 0.33130502629966674
lr = LogisticRegression(random_state=0)
lr.fit(X_tr,y_tr)
pred = lr.predict(X_val)
print(f1_score(y_val,pred,average='macro'))

0.33130502629966674


/Users/gihyun/Desktop/dev/GitHub/-BigDataAnalyst/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# 0.4368478801889553
dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_tr,y_tr)
pred = dt.predict(X_val)
print(f1_score(y_val,pred,average='macro'))

0.4368478801889553


In [208]:
# 0.5097607361203126
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr,y_tr)
pred = rf.predict(X_val)
print(f1_score(y_val,pred,average='macro'))

0.5097607361203126


In [212]:
pred = rf.predict(test)
submit = pd.DataFrame({
    'ID':test['ID'],
    'Segmentation':pred
})

### 데이터 전처리 및 심화




In [215]:
import pandas as pd

train = pd.read_csv("./ch4_data/train.csv")
test = pd.read_csv("./ch4_data/test.csv")
target = train.pop('Segmentation')

train.drop('ID',axis=1, inplace=True)
test_ID = test.pop('ID')

train = pd.get_dummies(train)
test = pd.get_dummies(test)

from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    train,
    target,
    test_size=0.2,
    random_state=0
)

X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)

rf.fit(X_tr,y_tr)
pred = rf.predict(X_val)
print(f1_score(y_val,pred,average='macro'))

0.4860743688384891


In [216]:
import pandas as pd

train = pd.read_csv("./ch4_data/train.csv")
test = pd.read_csv("./ch4_data/test.csv")
target = train.pop('Segmentation')

train.drop('ID',axis=1, inplace=True)
test_ID = test.pop('ID')

from sklearn.preprocessing import LabelEncoder
cols = train.select_dtypes(include='O').columns

for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    train,
    target,
    test_size=0.2,
    random_state=0
)

X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)

rf.fit(X_tr,y_tr)
pred = rf.predict(X_val)
print(f1_score(y_val,pred,average='macro'))

0.4940937547227707


In [218]:
import pandas as pd

train = pd.read_csv("./ch4_data/train.csv")
test = pd.read_csv("./ch4_data/test.csv")
target = train.pop('Segmentation')

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
cols =['ID','Age','Work_Experience','Family_Size']

train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])


train = pd.get_dummies(train)
test = pd.get_dummies(test)

from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    train,
    target,
    test_size=0.2,
    random_state=0
)

X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)

rf.fit(X_tr,y_tr)
pred = rf.predict(X_val)
print(f1_score(y_val,pred,average='macro'))

0.509334828690811


In [219]:
pred = rf.predict(test)
submit = pd.DataFrame({
    'ID':test['ID'],
    'Segmentation':pred
})

# 제 5회 기출문제


## 1유형


### 문제
종량제 봉투 종류가 '규격봉투'고 종량제 봉투 용도가 '음식물쓰레기'인 2L 가격 평균을 구하시오.<br>
단, 가격이 0이면 평균에서 제외하고, 반올림 후 정수를 출력하시오.

In [220]:
import pandas as pd

df = pd.read_csv('./ch5_data/data5-1.csv')

In [226]:
cond1 = df['종량제봉투종류']=='규격봉투'
cond2 = df['종량제봉투용도']=='음식물쓰레기'
cond3 = df['2ℓ가격']>0

df = df[cond1&cond2&cond3]

print(round(df['2ℓ가격'].mean()))

118


### 문제
bmi 수치가 정상인 사람 수와 위험체중인 사람 수의 차이를 절대값으로 구하시오(정수)


In [229]:
import pandas as pd
df = pd.read_csv('./ch5_data/data5-2.csv')
df['bmi'] = df['Weight'] / ((df['Height'] * 0.01)**2)

In [231]:
# 정상 체중
cond1 = (df['bmi'] >=18.5)&(df['bmi']<23)
cond2 = (df['bmi']>=23) & (df['bmi']<25)

print(abs(len(df[cond1])-len(df[cond2])))

144


### 문제
순 전입학생이 가장 많은 학교의 전체 학생수를 구하시오.<br>
(순 전입학생 = 전입학생 - 전출학생)

In [239]:
import pandas as pd
df = pd.read_csv('./ch5_data/data5-3.csv')
df['순 전입학생'] = df['전입학생수(계)']-df['전출학생수(계)']

In [241]:
df = df.sort_values('순 전입학생', ascending=False)

In [243]:
int(df.iloc[0,-2])

230

## 2유형

### 문제.
차량의 특징 기반 데이터를 활용해 test 데이터의 중고차 가격을 예측하시오.<br>

- 예측할 컬럼 : price
- 모델 성능 rmse(Root Mean Square Error) 평가지표에 따라 채점


In [253]:
import pandas as pd
train = pd.read_csv('./ch5_data/train.csv')
test = pd.read_csv('./ch5_data/test.csv')

In [254]:
train.shape, test.shape

((3759, 9), (1617, 8))

In [255]:
# 결측치 x
# 범주형 3개
# 수치형 6개
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3759 entries, 0 to 3758
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         3759 non-null   object 
 1   year          3759 non-null   int64  
 2   price         3759 non-null   int64  
 3   transmission  3759 non-null   object 
 4   mileage       3759 non-null   int64  
 5   fuelType      3759 non-null   object 
 6   tax           3759 non-null   int64  
 7   mpg           3759 non-null   float64
 8   engineSize    3759 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 264.4+ KB


In [256]:
# 결측치 x
# 범주형 3개
# 수치형 5개
test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1617 entries, 0 to 1616
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         1617 non-null   object 
 1   year          1617 non-null   int64  
 2   transmission  1617 non-null   object 
 3   mileage       1617 non-null   int64  
 4   fuelType      1617 non-null   object 
 5   tax           1617 non-null   int64  
 6   mpg           1617 non-null   float64
 7   engineSize    1617 non-null   float64
dtypes: float64(2), int64(3), object(3)
memory usage: 101.2+ KB


In [257]:
train.describe(include='O')

,model,transmission,fuelType
count,3759,3759,3759
unique,19,3,3
top,Fiesta,Manual,Petrol
freq,1372,3234,2541


In [258]:
target = train.pop('price')
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [259]:
from sklearn.model_selection import train_test_split

X_tr,X_val,y_tr,y_val = train_test_split(
    train,target, test_size=0.2, random_state=0
)

X_tr.shape, X_val.shape, y_tr.shape,y_val.shape

((3007, 30), (752, 30), (3007,), (752,))

In [264]:
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor


In [265]:
lr = LogisticRegression()
lr.fit(X_tr,y_tr)
pred = lr.predict(X_val)
print(root_mean_squared_error(y_val,pred))

4018.173528026109


/Users/gihyun/Desktop/dev/GitHub/-BigDataAnalyst/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [266]:
rf = RandomForestRegressor(random_state=0)
rf.fit(X_tr,y_tr)
pred = rf.predict(X_val)
print(root_mean_squared_error(y_val,pred))

1341.943732584459


#### 성능 개선

In [270]:
import pandas as pd
train = pd.read_csv("./ch5_data/train.csv")
test = pd.read_csv('./ch5_data/test.csv')

target = train.pop('price')

from sklearn.preprocessing import LabelEncoder
cols = train.select_dtypes(include='O').columns
for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    train, target, test_size =0.2, random_state=0
)

In [271]:
rf = RandomForestRegressor(random_state=0)
rf.fit(X_tr,y_tr)
pred = rf.predict(X_val)
print(root_mean_squared_error(y_val,pred))

1302.5271377093507


In [272]:
pred = rf.predict(test)
result = pd.DataFrame({
    'pred':pred
})

In [273]:
result

,pred
0,15622.74
1,15892.63
2,14940.32
3,18392.95
4,6164.23
...,...
1612,10887.99
1613,17342.15
1614,9529.61
1615,14371.82


# 제 6회 기출문제


## 제 1유형

### 문제
출동 시간과 도착시간 차이가 평균적으로 가장 오래 걸린 소방서의 시간을 분으로 변환해 출력하시오.

In [ ]:
import pandas as pd
df = pd.read_csv('./ch6_data/data6-1-1.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   소방서     100 non-null    object
 1   출동시간    100 non-null    object
 2   도착시간    100 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


In [276]:
df['출동시간'] = pd.to_datetime(df['출동시간'])
df['도착시간'] = pd.to_datetime(df['도착시간'])

In [278]:
df['diff'] = df['도착시간'] - df['출동시간']
df['diff'] = df['diff'].dt.total_seconds()/60

In [280]:
avg_diff = df.groupby('소방서')['diff'].mean()
idx = avg_diff.idxmax()

In [281]:
result = avg_diff[idx]
print(round(result))

81


### 문제
- 학교에서 교사 한 명당 맡은 학생수가 가장 많은 학교를 찾고, 그 학교의 전체 교사 수를 구하시오.

In [285]:
import pandas as pd
df = pd.read_csv("./ch6_data/data6-1-2.csv")
df['총학생수'] = df.iloc[:,2:].sum(axis=1)

In [286]:
df

,학교명,교사수,1학년,2학년,3학년,4학년,5학년,6학년,총학생수
0,학교1,20,118,72,154,118,77,59,598
1,학교2,31,187,193,135,137,63,90,805
2,학교3,23,136,193,135,112,91,50,717
3,학교4,32,72,146,67,51,130,164,630
4,학교5,22,61,73,86,82,55,124,481
...,...,...,...,...,...,...,...,...,...
95,학교96,38,75,166,113,191,172,57,774
96,학교97,30,125,169,145,122,187,194,942
97,학교98,37,136,100,93,60,145,63,597
98,학교99,37,74,175,145,157,95,76,722


In [288]:
df['학생/교사'] = df['총학생수']/df['교사수']

idx = df['학생/교사'].idxmax()

In [289]:
print(df.loc[idx,'교사수'])

19


### 문제
연도별로 총 범죄 건수(범죄 유형별 발생 건수의 합)의 월 평균 값을 구한 후 <br>
그 값이 가장 큰 연도를 찾아 해당 연도의 총 범죄 건수의 월평균 값을 출력

In [290]:
import pandas as pd
df = pd.read_csv("./ch6_data/data6-1-3.csv")

In [300]:
# df.iloc[:,1:-1]

df['총범죄수'] = df.iloc[:,1:-1].sum(axis=1)

In [301]:
df['연도'] = df['날짜'].str[:4]

In [303]:
result = df['총범죄수'].groupby(df['연도']).sum()
print(round(result.max()/12))

533


## 2유형

### 문제
건물 데이터를 기반으로 건물의 난방 부하(에너지 효율성)을 예측하시오.<br>
- 예측할 컬럼 : Heat_Load(Very Low,Low, Medium, High, Very High)
- 모델의 성능은 Macro F1-score평가지표에 따라 채점

In [304]:
import pandas as pd
train = pd.read_csv('./ch6_data/energy_train.csv')
test = pd.read_csv('./ch6_data/energy_test.csv')

train.shape, test.shape

((537, 10), (231, 9))

In [ ]:
train.info()
# 결측치 x
# 수치형 컬럼 : 6개
# 범주형 컬럼 : 4개

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537 entries, 0 to 536
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Compac       537 non-null    float64
 1   Surf_Area    537 non-null    float64
 2   Wall_Area    537 non-null    float64
 3   Roof         537 non-null    object 
 4   Height       537 non-null    object 
 5   Orient       537 non-null    object 
 6   Glaze_Area   537 non-null    float64
 7   Glaze_Distr  537 non-null    int64  
 8   Cool_Load    537 non-null    float64
 9   Heat_Load    537 non-null    object 
dtypes: float64(5), int64(1), object(4)
memory usage: 42.1+ KB


In [307]:
test.info()
# 결측치 x
# 수치형 컬럼 : 6개
# 범주형 컬럼 : 3개

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Compac       231 non-null    float64
 1   Surf_Area    231 non-null    float64
 2   Wall_Area    231 non-null    float64
 3   Roof         231 non-null    object 
 4   Height       231 non-null    object 
 5   Orient       231 non-null    object 
 6   Glaze_Area   231 non-null    float64
 7   Glaze_Distr  231 non-null    int64  
 8   Cool_Load    231 non-null    float64
dtypes: float64(5), int64(1), object(3)
memory usage: 16.4+ KB


In [311]:
train.describe()

,Compac,Surf_Area,Wall_Area,Glaze_Area,Glaze_Distr,Cool_Load
count,537.00000,537.000000,537.000000,537.000000,537.000000,537.000000
mean,0.76879,667.796089,318.271881,0.237244,2.808194,24.939628
std,0.10636,88.123635,42.447824,0.132183,1.541251,9.600170
min,0.62000,514.500000,245.000000,0.000000,0.000000,11.170000
25%,0.69000,588.000000,294.000000,0.100000,2.000000,15.770000
50%,0.76000,661.500000,318.500000,0.250000,3.000000,25.160000
75%,0.86000,735.000000,343.000000,0.400000,4.000000,33.540000
max,0.98000,808.500000,416.500000,0.400000,5.000000,48.030000


In [317]:
train.describe(include='O')

,Roof,Height,Orient,Heat_Load
count,537,537,537,537
unique,4,2,4,5
top,220.5,Tall,South,Very Low
freq,257,280,145,142


In [319]:
target = train.pop("Heat_Load")

In [320]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [321]:
from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    train,target,test_size=0.2, random_state=0
)

X_tr.shape, X_val.shape,y_tr.shape, y_val.shape

((429, 16), (108, 16), (429,), (108,))

In [323]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [ ]:
dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_tr,y_tr)
pred = dt.predict(X_val)
print(f1_score(y_val,pred, average='macro'))

0.9207614607614607


In [327]:
rt = RandomForestClassifier(random_state=0)
rt.fit(X_tr,y_tr)
pred = rt.predict(X_val)
print(f1_score(y_val, pred,average='macro'))


0.9277616846430405


In [329]:
pred = rt.predict(test)
submit = pd.DataFrame({
    'pred':pred
})

submit

,pred
0,Low
1,High
2,High
3,Low
4,Low
...,...
226,Very Low
227,Medium
228,Very Low
229,Low


## 3유형

### 문제
감기약 위약을 투여받은 환자의 부작용은 항암약 위약을 투여받은 환자의 부작용과 차이가 있는가?

1. 아픔 (비율 : 10%)
2. 조금 아픔 (비율 : 5%)
3. 속쓰림 (비율 : 15%)
4. 이상없음 (비율 : 70%)



In [1]:
import pandas as pd
df = pd.DataFrame({
    '항암약':[4,4,3,4,1,4,1,4,1,4,4,2,1,4,2,3,2,4,4,4]
})

print(df.head(3))

   항암약
0    4
1    4
2    3


- 귀무가설 : 감기약(위약)의 부작용과 항암약(위약)의 부작용은 동일하다.
- 대립가설 : 감기약(위약)의 부작용과 항암약(위약)의 부작용은 다르다.

1. 문제 항암약을 투여받은 환자 중 '이상없음'의 비율을 0과 1사이로 구하시오.
2. 감기약의 예상 부작용 비율과 항암약의 부작용 관찰값이 통게적으로 유의미하게 차이가 있는지 확인하려 한다.(카이제곱 검정을 사용해 검정 통계량을 구하시오.)
3. 위의 p-value를 구하시오

In [2]:
cnt = sum(df['항암약']==4)

ratio = cnt/len(df)
print(ratio)

0.55


In [5]:
# 1. 아픔 (비율 : 10%)
# 2. 조금 아픔 (비율 : 5%)
# 3. 속쓰림 (비율 : 15%)
# 4. 이상없음 (비율 : 70%)
from scipy.stats import chisquare
prob = [0.1,0.05, 0.15, 0.7]

print("데이터 수 : " ,len(df))

expected_counts = [0.1*20,0.05*20,0.15*20,0.7*20]
print(expected_counts)

expected_counts = [x*len(df) for x in prob]
print(expected_counts)

observed_counts = df['항암약'].value_counts().sort_index().to_list()
print(observed_counts)

print(chisquare(expected_counts,observed_counts))

데이터 수 :  20
[2.0, 1.0, 3.0, 14.0]
[2.0, 1.0, 3.0, 14.0]
[4, 3, 2, 11]
Power_divergenceResult(statistic=np.float64(3.6515151515151514), pvalue=np.float64(0.30163621963367865))


### 문제
- 독립변수 : solar(태양 에너지), wind(바람의 세기), O3(오존 농도)
- 종속변수 : temperature(온도)

In [7]:
import pandas as pd
df = pd.read_csv('./ch6_data/data6-3-2.csv')
df.head(3)

,solar,wind,o3,temperature
0,89.14,6.28,33.52,23.0
1,109.97,1.04,27.01,20.7
2,102.83,6.42,41.00,20.5


1. 주어진 데이터를 기반으로 다중 선형 회귀 모델을 구축하고, 독립변수 o3의 회귀계수를 구하시오
2. 문제 1에서 적합한 모델 'solar'와 o3이 고정된 상태에서 wind의 세기가 증가함에 따라 temperature가 감소한다.<br>
    이 회귀 모델에서 wind의 회귀계수에 대한 p-value 값을 구하시오.
3. 문제 2에서 적합한 모델에서 solar : 100, wind :5, o3:30일때 예측값을 구하시오.

In [10]:
from statsmodels.formula.api import ols

formula = 'temperature ~ solar + wind + o3'
model = ols(formula, df).fit()
summary = model.summary()
print(summary)

                            OLS Regression Results                            
Dep. Variable:            temperature   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     1.464
Date:                Sun, 23 Nov 2025   Prob (F-statistic):              0.229
Time:                        22:23:56   Log-Likelihood:                -195.45
No. Observations:                 100   AIC:                             398.9
Df Residuals:                      96   BIC:                             409.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     19.0507      1.994      9.555      0.0

In [12]:
print('1번 답 ', model.params['o3'])

1번 답  0.07493854378136566


In [13]:
print("2번 문제", model.pvalues['wind'])

2번 문제 0.779717720207169


In [15]:
new_data = pd.DataFrame({
    'solar' : [100], 'wind' :[5], 'o3':[30]
})

model.predict(new_data)

0    21.56163
dtype: float64

# 제 7회 기출문제

## 1유형


### 문제
결측치가 있는 행을 제거한 후 학생이 가장 많이 수강한 과목을 찾고, 해당 과목 점수를 표준화(스탠더드 스케일)한 다음 가장 큰 표준화 된 값을 구하시오.

In [42]:
import pandas as pd
df = pd.read_csv('./ch7_data/student_assessment.csv')
df

,id_assessment,student_id,study_period_days,score
0,103,0,22,87.0
1,93,1,70,92.0
2,15,2,22,2.0
3,107,3,22,51.0
4,72,4,22,73.0
...,...,...,...,...
2560,96,2695,35,3.0
2561,38,2696,22,77.0
2562,87,2697,22,88.0
2563,88,2698,70,67.0


In [43]:
df.dropna(inplace=True)

In [44]:
id = df['id_assessment'].value_counts().idxmax()


In [45]:
cond = df['id_assessment'] == id
df = df[cond]

In [48]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df['score'] = scaler.fit_transform(df[['score']])

/var/folders/6c/5zw1q8hx6r51frqls4xy7vdc0000gn/T/ipykernel_1511/427871623.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'] = scaler.fit_transform(df[['score']])


In [49]:
print(round(df['score'].max(),3))

2.183
